In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random                                #random song recommandation
from Levenshtein import ratio               # similarity ratio
from config import client_id, client_secret
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pprint
from IPython.display import IFrame

<h2> Dataframe Top_100 </h2>

In [ ]:
# 1. Get top 100 and create Dataframe with artists & titles

In [6]:
url = "https://www.billboard.com/charts/hot-100/"

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print(f"Error: {response.status_code}")

In [7]:
#artists
soup.find_all("span", class_="c-label")

[<span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	1
 </span>,
 <span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	18
 </span>,
 <span class="c-label a-font-primary-bold-xxl lrv-u-padding-lr-1 lrv-u-padding-t-025">
 	
 	31
 </span>,
 <span class="c-label a-font-primary-bold-l u-font-size-32@tablet u-letter-spacing-0080@tablet">
 	
 	1
 </span>,
 <span class="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet">
 	
 	Shaboozey
 </span>,
 <span class="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb-050@mobile-max u-font-size-32@tablet">
 	
 	1
 </span>,
 <span class="c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max lrv-u-padding-tb

In [12]:
artists=[]
for a in soup.select("span.c-label.a-no-trucate"):
    artists.append(a.get_text(strip=True))
artists

['Shaboozey',
 'Lady Gaga & Bruno Mars',
 'Billie Eilish',
 'Sabrina Carpenter',
 'Teddy Swims',
 'Gracie Abrams',
 'Post Malone Featuring Morgan Wallen',
 'Sabrina Carpenter',
 'Benson Boone',
 'Morgan Wallen',
 'Chappell Roan',
 'Hozier',
 'ROSE & Bruno Mars',
 'The Weeknd & Playboi Carti',
 'Sabrina Carpenter',
 'Kendrick Lamar',
 'Tyler, The Creator Featuring GloRilla, Sexyy Red & Lil Wayne',
 'Jelly Roll',
 'Morgan Wallen',
 'Myles Smith',
 'Chappell Roan',
 'Gigi Perez',
 'Tyler, The Creator Featuring Daniel Caesar',
 'Oscar Maydon & Fuerza Regida',
 'Billie Eilish',
 'GloRilla & Sexyy Red',
 'Gracie Abrams',
 'Tommy Richman',
 'Zach Bryan',
 'Sabrina Carpenter',
 'Teddy Swims',
 'Marshmello & Kane Brown',
 'Zach Bryan',
 'Koe Wetzel & Jessie Murph',
 'Ella Langley Featuring Riley Green',
 'GloRilla',
 'Morgan Wallen Featuring ERNEST',
 'Wham!',
 'Luke Combs',
 'Tucker Wetmore',
 'Taylor Swift',
 'Dasha',
 'Tyler, The Creator Featuring Teezo Touchdown',
 'Chappell Roan',
 'Jimin'

In [ ]:
#titles
#soup.find_all(class_ = 'o-chart-results-list-row-container')                # find all containers

In [20]:
titles = []
for title in soup.find_all(class_ = 'o-chart-results-list-row-container'):     #above h3
   titles.append(title.h3.get_text(strip = True))
titles

['A Bar Song (Tipsy)',
 'Die With A Smile',
 'Birds Of A Feather',
 'Espresso',
 'Lose Control',
 "That's So True",
 'I Had Some Help',
 'Taste',
 'Beautiful Things',
 'Love Somebody',
 'Good Luck, Babe!',
 'Too Sweet',
 'APT.',
 'Timeless',
 'Please Please Please',
 'Not Like Us',
 'Sticky',
 'I Am Not Okay',
 'Lies Lies Lies',
 'Stargazing',
 'Hot To Go!',
 'Sailor Song',
 'St. Chroma',
 'Tu Boda',
 'Wildflower',
 'Whatchu Kno About Me',
 "I Love You, I'm Sorry",
 'Million Dollar Baby',
 'High Road',
 'Bed Chem',
 'The Door',
 'Miles On It',
 'Pink Skies',
 'High Road',
 'You Look Like You Love Me',
 'TGIF',
 'Cowgirls',
 'Last Christmas',
 "Ain't No Love In Oklahoma",
 "Wind Up Missin' You",
 'I Can Do It With A Broken Heart',
 'Austin',
 'Darling, I',
 'Pink Pony Club',
 'Who',
 'Like Him',
 'Help Me',
 'Rah Tah Tah',
 "This World's A Giant",
 'Kehlani',
 'Hard Fought Hallelujah',
 'Dancing In The Flames',
 'Beautiful As You',
 'Dos Dias',
 "It's Ok I'm Ok",
 'Disease',
 '25',
 'I 

In [22]:
# create Dataframe 

df= pd.DataFrame({
    "Titles": titles,
    "Artists": artists})
df

,Titles,Artists
0,A Bar Song (Tipsy),Shaboozey
1,Die With A Smile,Lady Gaga & Bruno Mars
2,Birds Of A Feather,Billie Eilish
3,Espresso,Sabrina Carpenter
4,Lose Control,Teddy Swims
...,...,...
95,Prove It,21 Savage & Summer Walker
96,4x4xU,Lainey Wilson
97,I Ain't Sayin',Jordan Davis
98,The Largest,BigXthaPlug


In [24]:
#df.to_csv("df_hot_100.csv", index=False)

In [26]:
# Create a Python program that takes a user input, checks if it is present in your scraped DataFrame

In [28]:
print("You can type 'exit' at any time to quit.")
while True:
    user_input = input("Please enter a song: ").strip()
    # Exit Condition!
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    if user_input in df["Titles"].values:
        random_song = df.sample(1)["Titles"].iloc[0]                                  #randomly one row
        print(f"Your song is popular! Here's another suggestion: '{random_song}'")
    else:
        print("Sorry, your song is not hot enough.")

You can type 'exit' at any time to quit.


Please enter a song:  exit


Goodbye!


In [ ]:
# Handling Typos
#!pip install python-Levenshtein
"""The Levenshtein algorithm measures how many operations (insertion, deletion, or substitution) are required to transform one string into another.
The result is a similarity score ranging from 0 (no similarity) to 1 (identical). If the similarity of a title exceeds the current best_score, that title is stored as the new best_match."""

In [30]:
print("You can type 'exit' at any time to quit.")
while True:
    user_input = input("Please enter a song: ").strip()
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
                                                                # Levenshtein
    best_match = None
    best_score = 0
    
    for title in df["Titles"]:
        similarity = ratio(user_input, title)       # calculates similarity between 0 and 1
        if similarity > best_score:
            best_score = similarity
            best_match = title

    # check if similarity is high enough 
    if best_score >= 0.99:
        random_song = df.sample(1)["Titles"].iloc[0]
        print(f"It's hot! Here's is another suggestion: '{random_song}'")
    elif best_score >= 0.6:                                     # similarity treshhold: 60 % ?
        random_song = df.sample(1)["Titles"].iloc[0]  
        print(f"Did you mean '{best_match}'? It's in the list! Here's another suggestion: '{random_song}'")
    else:
        print("Sorry, your song is not hot enough.")

You can type 'exit' at any time to quit.


Please enter a song:  exit


Goodbye!


In [ ]:
# Embedded track player 

In [34]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [36]:
def get_spotify_id(title, artist):
    query = f"track:{title} artist:{artist}"
    result = sp.search(query, type="track", limit=1)
    if result['tracks']['items']:
        return result['tracks']['items'][0]['id']  # Track-ID
    else:
        return None  

In [40]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [42]:
print("You can type 'exit' at any time to quit.")
while True:
    user_input = input("Please enter a song: ").strip()
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    best_match = None
    best_score = 0
    best_artist = None

    for index, row in df.iterrows():
        title = row["Titles"]
        artist = row["Artists"]
        similarity = ratio(user_input.lower(), title.lower())  # Berechnet die Ähnlichkeit
        if similarity > best_score:
            best_score = similarity
            best_match = title
            best_artist = artist

  
    if best_score >= 0.99:  # Exakte Übereinstimmung
        track_id = get_spotify_id(best_match, best_artist)
        if track_id:
            print(f"Playing '{best_match}' by {best_artist}!")
            display(play_song(track_id))  # Song im Notebook abspielen
        else:
            print(f"Sorry, no Spotify link found for '{best_match}'.")
    elif best_score >= 0.6:  # Ähnliche Titel
        track_id = get_spotify_id(best_match, best_artist)
        if track_id:
            print(f"Did you mean '{best_match}' by {best_artist}? Playing it now!")
            display(play_song(track_id))  # Song im Notebook abspielen
        else:
            print(f"Sorry, no Spotify link found for '{best_match}'.")
    else:
        print("Sorry, your song is not hot enough.")

You can type 'exit' at any time to quit.


Please enter a song:  espreso


Did you mean 'Espresso' by Sabrina Carpenter? Playing it now!


Please enter a song:  exit


Goodbye!


<h2> Dataframe Audio_features </h2>

In [ ]:
# Using the Spotipy library, create a DataFrame storing the audio_features of at least 1000 songs.

In [ ]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [ ]:
# pprint.pprint(results)

In [ ]:
# Embedded track player

In [44]:
track_id = '3WMbD1OyfKuwWDWMNbPQ4g'                           # -> def play_song(track:_id)
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
# MOST POPULAR GENRES https://blog.novecore.com/top-10-most-popular-music-genres/

In [46]:
# Define get playlist function

def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
# 1. POP

In [ ]:
"""# find playlist id searching a playlist copy fromn url 
for playlist in result playlist itrms
    print playlist id 
results[playlist][items][0][id] """

In [ ]:
pop_playlist_tracks = get_playlist_tracks("Spotify","6mtYuOxzl58vSGnEDtZ9uB")

In [ ]:
pop_ids = []
for i in pop_playlist_tracks:
    pop_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
pop_ids_chopped = [pop_ids[i:i+chunk_size] for i in range(0,len(pop_ids), 100)]

In [ ]:
#pop_ids_chopped

In [ ]:
pop_af = []
for i in pop_ids_chopped:
    pop_af.extend(sp.audio_features(i))
pd.DataFrame(pop_af)

In [ ]:
pop_af = pd.DataFrame(pop_af)
#pop_af.to_csv("pop.csv", index=False) 

In [ ]:
# 2. ROCK 

In [ ]:
rock_playlist_tracks = get_playlist_tracks("Spotify","3sTZTkIGgm8wJiSXDvpApF")

In [ ]:
rock_ids = []
for i in rock_playlist_tracks:
    rock_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
rock_ids_chopped = [rock_ids[i:i+chunk_size] for i in range(0,len(rock_ids), 100)]

In [ ]:
rock_af = []
for i in rock_ids_chopped:
    rock_af.extend(sp.audio_features(i))
pd.DataFrame(rock_af).head(3)

In [ ]:
rock_af = pd.DataFrame(rock_af)
#rock_af.to_csv("rock.csv", index=False) 

In [ ]:
# 3. HipHop

In [ ]:
hop_playlist_tracks = get_playlist_tracks("Spotify","56un2laj6rmMUKhDlkUkAY")

In [ ]:
hop_ids = []
for i in hop_playlist_tracks:
    hop_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
hop_ids_chopped = [hop_ids[i:i+chunk_size] for i in range(0,len(hop_ids), 100)]

In [ ]:
hop_af = []
for i in hop_ids_chopped:
    hop_af.extend(sp.audio_features(i))
pd.DataFrame(hop_af).head(3)

In [ ]:
hop_af = pd.DataFrame(hop_af)
#hop_af.to_csv("hop.csv", index=False) 

In [ ]:
# 4. R&B 

In [ ]:
rb_playlist_tracks = get_playlist_tracks("Spotify","3a9yodm1vqcU6JULLBeZeA")

In [ ]:
rb_ids = []
for i in rb_playlist_tracks:
    rb_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
rb_ids_chopped = [rb_ids[i:i+chunk_size] for i in range(0,len(hop_ids), 100)]

In [ ]:
rb_af = []
for i in rb_ids_chopped:
    rb_af.extend(sp.audio_features(i))
pd.DataFrame(rb_af).head(3)

In [ ]:
rb_af = pd.DataFrame(rb_af)
#rb_af.to_csv("rb.csv", index=False) 

In [ ]:
# 5. EDM

In [ ]:
edm_playlist_tracks = get_playlist_tracks("Spotify","1lS6v9h4MXOw6f6y8MkS8w")

In [ ]:
edm_ids = []
for i in edm_playlist_tracks:
    edm_ids.append(i['track']['id'])

In [ ]:
edm_af = []
for i in edm_ids:
    edm_af.extend(sp.audio_features(i))
pd.DataFrame(edm_af).head(3)

In [ ]:
edm_af = pd.DataFrame(edm_af)
#edm_af.to_csv("edm.csv", index=False) 

In [ ]:
# 6. Country 

In [ ]:
cn_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DWZBCPUIUs2iR")

In [ ]:
cn_ids = []
for i in cn_playlist_tracks:
    cn_ids.append(i['track']['id'])

In [ ]:
cn_af = []
for i in cn_ids:
    cn_af.extend(sp.audio_features(i))
pd.DataFrame(cn_af).head(3)

In [ ]:
cn_af = pd.DataFrame(cn_af)
#cn_af.to_csv("cn.csv", index=False) 

In [ ]:
# 7. Jazz 

In [ ]:
jz_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DXbITWG1ZJKYt")

In [ ]:
jz_ids = []
for i in jz_playlist_tracks:
    jz_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
jz_ids_chopped = [jz_ids[i:i+chunk_size] for i in range(0,len(jz_ids), 100)]

In [ ]:
jz_af = []
for i in jz_ids_chopped:
    jz_af.extend(sp.audio_features(i))
pd.DataFrame(jz_af).head(3)

In [ ]:
jz_af = pd.DataFrame(jz_af)
#jz_af.to_csv("jz.csv", index=False) 

In [ ]:
# 8. Raeggae 

In [ ]:
rg_playlist_tracks = get_playlist_tracks("Spotify","0uKEs5Je7unyIpYlK9d320")

In [ ]:
rg_ids = []
for i in rg_playlist_tracks:
    rg_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
rg_ids_chopped = [rg_ids[i:i+chunk_size] for i in range(0,len(rg_ids), 100)]

In [ ]:
rg_af = []
for i in rg_ids_chopped:
    rg_af.extend(sp.audio_features(i))
pd.DataFrame(rg_af).head(3)

In [ ]:
rg_af = pd.DataFrame(rg_af)
#rg_af.to_csv("rg.csv", index=False) 

In [ ]:
# 9. Latin

In [ ]:
ln_playlist_tracks = get_playlist_tracks("Spotify","47lKGlxoijn8lJyWcNBwEW")

In [ ]:
ln_ids = []
for i in ln_playlist_tracks:
    ln_ids.append(i['track']['id'])

In [ ]:
ln_af = []
for i in ln_ids:
    ln_af.extend(sp.audio_features(i))
pd.DataFrame(ln_af).head(3)

In [ ]:
ln_af = pd.DataFrame(ln_af)
#ln_af.to_csv("ln.csv", index=False) 

In [ ]:
# 10. Classical 

In [ ]:
cl_playlist_tracks = get_playlist_tracks("Spotify","1DSEi0lvLOzTiKqnaYZBDh")

In [ ]:
cl_ids = []
for i in cl_playlist_tracks:
    cl_ids.append(i['track']['id'])

In [ ]:
cl_af = []
for i in cl_ids:
    cl_af.extend(sp.audio_features(i))
pd.DataFrame(cl_af).head(3)

In [ ]:
cl_af = pd.DataFrame(cl_af)
#cl_af.to_csv("cl.csv", index=False) 

In [ ]:
# 11. Techno

In [ ]:
tn_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DX6J5NfMJS675")

In [ ]:
tn_ids = []
for i in tn_playlist_tracks:
    tn_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
tn_ids_chopped = [tn_ids[i:i+chunk_size] for i in range(0,len(tn_ids), 100)]

In [ ]:
tn_af = []
for i in tn_ids_chopped:
    tn_af.extend(sp.audio_features(i))
pd.DataFrame(tn_af).head(3)

In [ ]:
tn_af = pd.DataFrame(tn_af)
#tn_af.to_csv("tn.csv", index=False) 

In [ ]:
# 12. Deutschrap

In [ ]:
dr_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DWSzguhfGl55y")

In [ ]:
dr_ids = []
for i in dr_playlist_tracks:
    dr_ids.append(i['track']['id'])

In [ ]:
dr_af = []
for i in dr_ids:
    dr_af.extend(sp.audio_features(i))
pd.DataFrame(dr_af).head(3)

In [ ]:
dr_af = pd.DataFrame(dr_af)
#dr_af.to_csv("dr.csv", index=False) 

In [ ]:
# 13. Metal

In [ ]:
mt_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1EQpgT26jgbgRI")

In [ ]:
mt_ids = []
for i in mt_playlist_tracks:
    mt_ids.append(i['track']['id'])

In [ ]:
mt_af = []
for i in mt_ids:
    mt_af.extend(sp.audio_features(i))
pd.DataFrame(mt_af).head(3)

In [ ]:
mt_af = pd.DataFrame(mt_af)
#mt_af.to_csv("mt.csv", index=False) 

In [ ]:
# 14. Indie

In [ ]:
ind_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DWYJeWl6ior4d")

In [ ]:
ind_ids = []
for i in ind_playlist_tracks:
    ind_ids.append(i['track']['id'])

In [ ]:
ind_af = []
for i in ind_ids:
    ind_af.extend(sp.audio_features(i))
pd.DataFrame(ind_af).head(3)

In [ ]:
ind_af = pd.DataFrame(ind_af)
#ind_af.to_csv("ind.csv", index=False) 

In [ ]:
# 15 Downtempo

In [ ]:
dt_playlist_tracks = get_playlist_tracks("Spotify","57uN4exPbSwPc4K3Fwx1vw")

In [ ]:
dt_ids = []
for i in dt_playlist_tracks:
    dt_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
dt_ids_chopped = [dt_ids[i:i+chunk_size] for i in range(0,len(dt_ids), 100)]

In [ ]:
dt_af = []
for i in dt_ids_chopped:
    dt_af.extend(sp.audio_features(i))
pd.DataFrame(dt_af).head(3)

In [ ]:
dt_af = pd.DataFrame(dt_af)
#dt_af.to_csv("dt.csv", index=False) 

In [ ]:
# 16 Soundtracks

In [ ]:
st_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DX8C9xQcOrE6T")

In [ ]:
st_ids = []
for i in st_playlist_tracks:
    st_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
st_ids_chopped = [st_ids[i:i+chunk_size] for i in range(0,len(st_ids), 100)]

In [ ]:
st_af = []
for i in st_ids_chopped:
    st_af.extend(sp.audio_features(i))
pd.DataFrame(st_af).head(3)

In [ ]:
st_af = pd.DataFrame(st_af)
#st_af.to_csv("st.csv", index=False) 

In [ ]:
# 17. mixed playlist 

In [ ]:
mx_playlist_tracks = get_playlist_tracks("Spotify","3ARGWPi5zf8sfl86RhG1UB")

In [ ]:
mx_ids = []
for i in mx_playlist_tracks:
    mx_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
mx_ids_chopped = [mx_ids[i:i+chunk_size] for i in range(0,len(mx_ids), 100)]

In [ ]:
mx_af = []
for i in mx_ids_chopped:
    mx_af.extend(sp.audio_features(i))
pd.DataFrame(mx_af).head(3)

In [ ]:
mx_af = pd.DataFrame(mx_af)
#mx_af.to_csv("mx.csv", index=False) 

In [ ]:
# 18. Oldies

In [ ]:
old_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DX56bqlsMxJYR")

In [ ]:
old_ids = []
for i in old_playlist_tracks:
    old_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
old_ids_chopped = [old_ids[i:i+chunk_size] for i in range(0,len(old_ids), 100)]

In [ ]:
old_af = []
for i in old_ids_chopped:
    old_af.extend(sp.audio_features(i))
pd.DataFrame(old_af).head(3)

In [ ]:
old_af = pd.DataFrame(old_af)
#old_af.to_csv("old.csv", index=False) 

In [ ]:
# 19. New

In [ ]:
new_playlist_tracks = get_playlist_tracks("Spotify","37i9dQZF1DX4JAvHpjipBk")

In [ ]:
new_ids = []
for i in new_playlist_tracks:
    new_ids.append(i['track']['id'])

In [ ]:
new_af = []
for i in new_ids:
    new_af.extend(sp.audio_features(i))
pd.DataFrame(new_af).head(3)

In [ ]:
new_af = pd.DataFrame(new_af)
#new_af.to_csv("new.csv", index=False) 

In [ ]:
# 20. Mixed2024

In [ ]:
m2_playlist_tracks = get_playlist_tracks("Spotify","2DpRAKsc8kVq3GaZ9LKaoC")

In [ ]:
m2_ids = []
for i in m2_playlist_tracks:
    m2_ids.append(i['track']['id'])

In [ ]:
chunk_size = 100
m2_ids_chopped = [m2_ids[i:i+chunk_size] for i in range(0,len(m2_ids), 100)]

In [ ]:
m2_af = []
for i in m2_ids_chopped:
    m2_af.extend(sp.audio_features(i))
pd.DataFrame(m2_af).head(3)

In [ ]:
m2_af = pd.DataFrame(m2_af)
#m2_af.to_csv("m2.csv", index=False) 

In [ ]:
# Combine Dataframes

In [ ]:
dataframes = [pop_af, rock_af, hop_af, rb_af, edm_af, cn_af, jz_af, rg_af, ln_af, cl_af, tn_af, dr_af, mt_af, ind_af, dt_af, st_af, mx_af, old_af, new_af, m2_af]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df

In [ ]:
#combined_df.duplicated().sum()
#combined_df = combined_df.drop_duplicates()

In [ ]:
#combined_df.to_csv("combined_df.csv", index=False) 

In [ ]:
combined_df.columns    #https://developer.spotify.com/documentation/web-api/reference/get-audio-features 

In [ ]:
combined_df

In [51]:
file_path='/Users/ironhack/Documents/GitHub/Song-Recommandation/combined_df.csv'
combined_df = pd.read_csv(file_path)
combined_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.623,0.734,9,-5.948,1,0.1070,0.01620,0.000002,0.1450,0.370,107.853,audio_features,76FZM38RC8XaAjJ77CVTNe,spotify:track:76FZM38RC8XaAjJ77CVTNe,https://api.spotify.com/v1/tracks/76FZM38RC8Xa...,https://api.spotify.com/v1/audio-analysis/76FZ...,244453,4
1,0.722,0.851,7,-3.873,0,0.0639,0.03190,0.000000,0.1080,0.452,126.885,audio_features,5yDL13y5giogKs2fSNf7sj,spotify:track:5yDL13y5giogKs2fSNf7sj,https://api.spotify.com/v1/tracks/5yDL13y5giog...,https://api.spotify.com/v1/audio-analysis/5yDL...,197936,4
2,0.647,0.585,6,-6.123,1,0.0512,0.00314,0.000000,0.1650,0.353,131.934,audio_features,5jrdCoLpJSvHHorevXBATy,spotify:track:5jrdCoLpJSvHHorevXBATy,https://api.spotify.com/v1/tracks/5jrdCoLpJSvH...,https://api.spotify.com/v1/audio-analysis/5jrd...,215672,4
3,0.688,0.734,9,-4.569,1,0.0274,0.00462,0.000000,0.0756,0.460,140.004,audio_features,2hns6Dv29Yrg68AVTJiAyA,spotify:track:2hns6Dv29Yrg68AVTJiAyA,https://api.spotify.com/v1/tracks/2hns6Dv29Yrg...,https://api.spotify.com/v1/audio-analysis/2hns...,208493,4
4,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,0.661,95.010,audio_features,7BKLCZ1jbUBVqRi2FVlTVw,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,https://api.spotify.com/v1/tracks/7BKLCZ1jbUBV...,https://api.spotify.com/v1/audio-analysis/7BKL...,244960,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,0.393,0.714,11,-8.521,1,0.0374,0.15900,0.000002,0.3120,0.769,115.522,audio_features,0mwr3py9MZIOdvOrI31h6r,spotify:track:0mwr3py9MZIOdvOrI31h6r,https://api.spotify.com/v1/tracks/0mwr3py9MZIO...,https://api.spotify.com/v1/audio-analysis/0mwr...,166520,4
1870,0.552,0.313,5,-12.410,1,0.0296,0.91600,0.001700,0.1200,0.303,88.554,audio_features,3YdKJzcoMZMacISlpY4QoP,spotify:track:3YdKJzcoMZMacISlpY4QoP,https://api.spotify.com/v1/tracks/3YdKJzcoMZMa...,https://api.spotify.com/v1/audio-analysis/3YdK...,202773,3
1871,0.592,0.522,2,-9.922,0,0.2170,0.77700,0.000000,0.2190,0.808,150.452,audio_features,0QoV9nLBwYEU1bA6m61h4F,spotify:track:0QoV9nLBwYEU1bA6m61h4F,https://api.spotify.com/v1/tracks/0QoV9nLBwYEU...,https://api.spotify.com/v1/audio-analysis/0QoV...,149560,4
1872,0.431,0.643,10,-11.391,1,0.3860,0.78500,0.000000,0.3600,0.540,201.611,audio_features,4rc5ZzvU4Ke8Yy2A2wRaIl,spotify:track:4rc5ZzvU4Ke8Yy2A2wRaIl,https://api.spotify.com/v1/tracks/4rc5ZzvU4Ke8...,https://api.spotify.com/v1/audio-analysis/4rc5...,142453,4
